# **ADD IDEOLOGY SCORES**



In [23]:
import pandas as pd
from collections import Counter
from scipy.stats import norm
import json
import seaborn as sns

In [24]:
model = "finetuned"
#model = "self"

#scores = "clipped"
scores = "non_clipped"

order = "len_EMI_z_trans"
#oder = "len_norm_z_trans_EMI"

In [25]:
subreddits = ["Ask_Politics", "Askpolitics", "NeutralPolitics", "PoliticalDebate", "PoliticalDiscussion", "politics"]

## **Interactions**

In [26]:
# data with correct labels

EMI_df = pd.read_csv(f"../data/analysis_data/df_all_{scores}.csv")

/tmp/ipykernel_14310/876061152.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  EMI_df = pd.read_csv(f"../data/analysis_data/df_all_{scores}.csv")


In [27]:
EMI_df[EMI_df.isnull().any(axis=1)]

,msg_id_parent,msg_id_child,submission_id,subreddit,exact_time_child,author_child,author_parent,author_submission,finetuned_child_E_norm,finetuned_child_I_norm,...,finetuned_parent_len_norm_z_trans_EMI,finetuned_child_len_norm_z_trans_EMI,finetuned_submission_len_norm_z_trans_EMI,self_parent_len_norm_z_trans_EMI,self_child_len_norm_z_trans_EMI,self_submission_len_norm_z_trans_EMI,predictions_label_ft,predictions_score_ft,predictions_prob_disagree,predictions_ft
23322,dndrvjw,dng4u4l,71v8rr,Ask_Politics,1506276675.0,Ronald-Hapchwarae,HiFidelityCastro,PrecookedEagle,NaN,NaN,...,-1.369760,NaN,-0.768807,-0.890489,NaN,-0.433912,0,0.9526,0.04745,no_disagreement
31933,en6fg2g,en6gjiw,bmetye,Ask_Politics,1557630434.0,OxymoronicallyAbsurd,Quetzalcoatls,OxymoronicallyAbsurd,0.101755,0.096608,...,0.859292,0.138972,NaN,0.781981,-0.132313,NaN,0,0.9814,0.01843,no_disagreement
42539,fy9qj9m,fya1dqt,hsbxhb,Ask_Politics,1594924260.0,floating_bells_down,LonelyNarwhal,floating_bells_down,NaN,NaN,...,-0.403764,NaN,-0.229096,-0.281348,NaN,-0.543555,0,0.9940,0.00574,no_disagreement
72871,c5e1xcj,c5e3aop,wk4bv,PoliticalDiscussion,1342306631.0,Impulse97,rustyarrowhead,TheMentatBashar,NaN,NaN,...,-0.386899,NaN,-0.825575,-0.239010,NaN,-1.028762,0,0.7856,0.21410,no_disagreement
84036,c6qj5tk,c6qjfgb,11y2ua,PoliticalDiscussion,1351004371.0,newpolitics,txking12,newpolitics,-0.142458,-0.289626,...,NaN,1.119183,0.149322,NaN,1.138499,0.456391,0,0.9795,0.02045,no_disagreement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4848952,jezpelx,jezpij3,12bqlrc,politics,1970-01-01 00:00:01.680655316,CardMechanic,poky2017,PoliticsModeratorBot,NaN,NaN,...,1.368682,NaN,-0.309337,1.611648,NaN,-0.050898,0,0.9300,0.06964,no_disagreement
4904149,k3bu54n,k3bub2y,16yy9i6,politics,1970-01-01 00:00:01.696362558,Lucky-Earther,szboy422,PoliticsModeratorBot,NaN,NaN,...,0.368310,NaN,-0.323166,0.926728,NaN,-0.014768,0,0.5460,0.45400,no_disagreement
4908130,k3chis1,k3chugj,16z3s32,politics,1970-01-01 00:00:01.696370677,HeLooks2Muuuch,NateGrey,PoliticsModeratorBot,NaN,NaN,...,0.795725,NaN,0.548592,0.765175,NaN,0.686053,0,0.6953,0.30470,no_disagreement
4912801,k5a3ejc,k5a3ib5,17a06s3,politics,1970-01-01 00:00:01.697561611,SpaceElevatorMusic,worriedjacket,PoliticsModeratorBot,NaN,NaN,...,-1.295026,NaN,0.136453,-0.957208,NaN,0.594228,1,0.8650,0.86500,disagree


In [28]:
EMI_df['author_child'] = EMI_df['author_child'].apply(lambda x: x.lower())
EMI_df['author_parent'] = EMI_df['author_parent'].apply(lambda x: x.lower())
EMI_df['author_submission'] = EMI_df['author_submission'].apply(lambda x: x.lower())

## **Texts**

In [29]:

parents = EMI_df[['msg_id_parent', 'author_parent',
                  'finetuned_parent_len_EMI_z_trans',
                  'finetuned_parent_len_norm_z_trans_EMI',
                  'self_parent_len_EMI_z_trans',
                  'self_parent_len_norm_z_trans_EMI',
                  'predictions_score_ft','predictions_ft', 'predictions_prob_disagree']].drop_duplicates(subset = 'msg_id_parent').reset_index(drop = True)



parents = parents.rename(columns = {'msg_id_parent': 'id', 'author_parent' : 'author',
                                    'finetuned_parent_len_EMI_z_trans' : 'finetuned_len_EMI_z_trans' ,
                                    'finetuned_parent_len_norm_z_trans_EMI' : 'finetuned_len_norm_z_trans_EMI',
                                    'self_parent_len_EMI_z_trans' : 'self_len_EMI_z_trans',
                                    'self_parent_len_norm_z_trans_EMI' : 'self_len_norm_z_trans_EMI'})


parents['type'] = 'parent'

# ---------------------------------------------------
children = EMI_df[['msg_id_child', 'author_child', 
                   'finetuned_child_len_EMI_z_trans',
                   'finetuned_child_len_norm_z_trans_EMI',
                   'self_child_len_EMI_z_trans',
                   'self_child_len_norm_z_trans_EMI',
                   'predictions_score_ft','predictions_ft', 'predictions_prob_disagree']].drop_duplicates(subset = 'msg_id_child').reset_index(drop = True)


children = children.rename(columns = {'msg_id_child': 'id',  'author_child' : 'author', 
                                      'finetuned_child_len_EMI_z_trans' : 'finetuned_len_EMI_z_trans' ,
                                      'finetuned_child_len_norm_z_trans_EMI' : 'finetuned_len_norm_z_trans_EMI',
                                      'self_child_len_EMI_z_trans' : 'self_len_EMI_z_trans',
                                      'self_child_len_norm_z_trans_EMI' : 'self_len_norm_z_trans_EMI'})


children['type'] = 'child'


# ---------------------------------------------------
submissions = EMI_df[['submission_id', 'author_submission', 
                      'finetuned_submission_len_EMI_z_trans',
                      'finetuned_submission_len_norm_z_trans_EMI',
                      'self_submission_len_EMI_z_trans',
                      'self_submission_len_norm_z_trans_EMI',
                      'predictions_score_ft','predictions_ft', 'predictions_prob_disagree']].drop_duplicates(subset = 'submission_id').reset_index(drop = True)


submissions = submissions.rename(columns = {'submission_id': 'id', 'author_submission' : 'author', 
                                            'finetuned_submission_len_EMI_z_trans' : 'finetuned_len_EMI_z_trans' ,
                                            'finetuned_submission_len_norm_z_trans_EMI' : 'finetuned_len_norm_z_trans_EMI',
                                            'self_submission_len_EMI_z_trans' : 'self_len_EMI_z_trans',
                                            'self_submission_len_norm_z_trans_EMI' : 'self_len_norm_z_trans_EMI'})


submissions['type'] = 'submission'



texts_EMI = pd.concat([parents, children, submissions]).dropna().reset_index(drop = True)
texts_EMI

,id,author,finetuned_len_EMI_z_trans,finetuned_len_norm_z_trans_EMI,self_len_EMI_z_trans,self_len_norm_z_trans_EMI,predictions_score_ft,predictions_ft,predictions_prob_disagree,type
0,c7ecp8d,ramblin_dash,0.476542,0.514532,0.711538,0.624110,0.6180,no_disagreement,0.38180,parent
1,c7efx8q,senatorskeletor,0.269695,0.269966,0.150568,0.130100,0.9507,disagree,0.95070,parent
2,c7edr13,tehnoff,-1.058486,-0.620579,-1.365538,-1.228132,0.8633,no_disagreement,0.13650,parent
3,c7ee45j,ibiteyou,0.249878,0.296310,0.241266,0.203621,0.9126,no_disagreement,0.08750,parent
4,c7ejopx,dublinben,-0.921904,-0.907752,-0.704820,-0.632541,0.9395,no_disagreement,0.06036,parent
...,...,...,...,...,...,...,...,...,...,...
7017396,18mfx16,politicsmoderatorbot,0.521842,0.396905,0.422541,0.240426,0.7163,no_disagreement,0.28370,submission
7017397,18mfyba,politicsmoderatorbot,0.521842,0.396905,0.422541,0.240426,0.9785,no_disagreement,0.02129,submission
7017398,18p9iet,optimalg,-0.633410,-0.580210,-0.732976,-0.857240,0.6665,disagree,0.66650,submission
7017399,18qmiau,optimalg,-0.548816,-0.584471,-0.369686,-0.645052,0.7540,no_disagreement,0.24610,submission


## **Add User Ideology**

In [30]:
import os
os.getcwd()

'/home/jovyan/user_ideology'

In [31]:
user_scores = pd.read_csv("user_positioning.csv")
user_scores

,User,Partisan_Score
0,mukhasim,-0.025850
1,fiplefip,0.040918
2,bobcat,-0.010514
3,mothramantra,-0.001673
4,postnationalism,0.022297
...,...,...
804509,the_general0186,-0.062967
804510,theenightmancometh,0.037424
804511,thegassykid,-0.055808
804512,the_end_of_memes,0.020002


In [32]:
user_scores[user_scores.isna().any(axis = 1)]

,User,Partisan_Score
42859,jhinn329,NaN
95449,citizen_concentrated,NaN
258040,ademergency2023,NaN
331900,kt0998,NaN
361636,square-vanilla-4478,NaN
...,...,...
804204,dull_mirror_163,NaN
804212,dudester101,NaN
804312,theclayboybunn,NaN
804346,theearthisrising,NaN


In [33]:
user_scores = user_scores.dropna().reset_index(drop = True)
user_scores

,User,Partisan_Score
0,mukhasim,-0.025850
1,fiplefip,0.040918
2,bobcat,-0.010514
3,mothramantra,-0.001673
4,postnationalism,0.022297
...,...,...
802596,the_general0186,-0.062967
802597,theenightmancometh,0.037424
802598,thegassykid,-0.055808
802599,the_end_of_memes,0.020002


## Find all texts (incl. submissions), for which I have ideology

In [34]:
texts_ideology = texts_EMI[texts_EMI["author"].isin(user_scores["User"])].reset_index(drop = True)
texts_ideology

texts_ideology = texts_ideology.merge(user_scores, how = 'left', left_on = "author", right_on="User").drop(columns = "User")
texts_ideology

,id,author,finetuned_len_EMI_z_trans,finetuned_len_norm_z_trans_EMI,self_len_EMI_z_trans,self_len_norm_z_trans_EMI,predictions_score_ft,predictions_ft,predictions_prob_disagree,type,Partisan_Score
0,c7ecp8d,ramblin_dash,0.476542,0.514532,0.711538,0.624110,0.6180,no_disagreement,0.38180,parent,0.017145
1,c7efx8q,senatorskeletor,0.269695,0.269966,0.150568,0.130100,0.9507,disagree,0.95070,parent,0.013507
2,c7edr13,tehnoff,-1.058486,-0.620579,-1.365538,-1.228132,0.8633,no_disagreement,0.13650,parent,0.035521
3,c7ee45j,ibiteyou,0.249878,0.296310,0.241266,0.203621,0.9126,no_disagreement,0.08750,parent,0.182155
4,c7ejopx,dublinben,-0.921904,-0.907752,-0.704820,-0.632541,0.9395,no_disagreement,0.06036,parent,0.024481
...,...,...,...,...,...,...,...,...,...,...,...
7010754,18letrx,optimalg,-0.510420,-0.555166,-0.326702,-0.625037,0.7837,no_disagreement,0.21620,submission,-0.024883
7010755,18m5lyo,19thnews,-0.002839,-0.084226,0.094693,-0.176572,0.9240,no_disagreement,0.07610,submission,-0.082393
7010756,18p9iet,optimalg,-0.633410,-0.580210,-0.732976,-0.857240,0.6665,disagree,0.66650,submission,-0.024883
7010757,18qmiau,optimalg,-0.548816,-0.584471,-0.369686,-0.645052,0.7540,no_disagreement,0.24610,submission,-0.024883


In [35]:
len(texts_ideology) / len(texts_EMI)


0.9990534957315393

## Interactions, I have idelogy for

In [36]:
child_found = EMI_df[EMI_df["author_child"].isin(user_scores["User"])]

interactions_ideology = child_found[child_found["author_parent"].isin(user_scores["User"])].reset_index(drop = True)
interactions_ideology

,msg_id_parent,msg_id_child,submission_id,subreddit,exact_time_child,author_child,author_parent,author_submission,finetuned_child_E_norm,finetuned_child_I_norm,...,finetuned_parent_len_norm_z_trans_EMI,finetuned_child_len_norm_z_trans_EMI,finetuned_submission_len_norm_z_trans_EMI,self_parent_len_norm_z_trans_EMI,self_child_len_norm_z_trans_EMI,self_submission_len_norm_z_trans_EMI,predictions_label_ft,predictions_score_ft,predictions_prob_disagree,predictions_ft
0,c7ecp8d,c7ecs2n,14m325,Ask_Politics,1355166551.0,karmanaut,ramblin_dash,karmanaut,0.053025,0.072843,...,0.514532,-0.116730,0.594431,0.624110,-0.011457,0.172262,0,0.6180,0.38180,no_disagreement
1,c7ecp8d,c7ecxpk,14m325,Ask_Politics,1355167049.0,zoolander951,ramblin_dash,karmanaut,0.040605,0.001874,...,0.514532,0.363800,0.594431,0.624110,0.012864,0.172262,0,0.7134,0.28640,no_disagreement
2,c7ecp8d,c7eflag,14m325,Ask_Politics,1355175187.0,zossima,ramblin_dash,karmanaut,0.037034,-0.026841,...,0.514532,0.571814,0.594431,0.624110,0.600814,0.172262,0,0.6580,0.34180,no_disagreement
3,c7efx8q,c7ejh1d,14m325,Ask_Politics,1355187644.0,fathermocker,senatorskeletor,karmanaut,0.046751,0.040446,...,0.269966,0.096978,0.594431,0.130100,0.070296,0.172262,1,0.9507,0.95070,disagree
4,c7efx8q,c7f8140,14m325,Ask_Politics,1355286254.0,pkmkii,senatorskeletor,karmanaut,0.029991,-0.043360,...,0.269966,0.644853,0.594431,0.130100,0.612353,0.172262,0,0.8580,0.14180,no_disagreement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4933850,kfkh5ib,kfmd52k,18uirzt,politics,1970-01-01 00:00:01.703977996,knightforlife,freddiethebaer,optimalg,0.289315,0.222072,...,1.250681,0.671616,-0.580210,1.062374,0.498045,-0.857240,0,0.9550,0.04495,no_disagreement
4933851,kfklsb6,kforzbx,18uirzt,politics,1970-01-01 00:00:01.704023024,bakerfredricka,4blockhead,optimalg,0.192579,0.210383,...,1.367783,-0.043416,-0.580210,1.165189,-0.381238,-0.857240,0,0.9920,0.00774,no_disagreement
4933852,kfmq5kr,kfp6tzv,18uirzt,politics,1970-01-01 00:00:01.704032059,glassjar1,doubletfan,optimalg,0.078374,0.101258,...,-0.197753,-0.140261,-0.580210,0.242695,-0.122822,-0.857240,1,0.8500,0.85000,disagree
4933853,kfp3d4v,kfp8xh8,18uirzt,politics,1970-01-01 00:00:01.704033069,haulinboats,4blockhead,optimalg,0.137089,0.224533,...,0.600514,-0.617072,-0.580210,0.207855,-0.789836,-0.857240,0,0.9414,0.05875,no_disagreement


In [37]:
child_found = EMI_df[EMI_df["author_child"].isin(user_scores["User"])]

parent_found = child_found[child_found["author_parent"].isin(user_scores["User"])].reset_index(drop = True)

full_interactions_ideology = parent_found[parent_found["author_submission"].isin(user_scores["User"])].reset_index(drop = True)
full_interactions_ideology

,msg_id_parent,msg_id_child,submission_id,subreddit,exact_time_child,author_child,author_parent,author_submission,finetuned_child_E_norm,finetuned_child_I_norm,...,finetuned_parent_len_norm_z_trans_EMI,finetuned_child_len_norm_z_trans_EMI,finetuned_submission_len_norm_z_trans_EMI,self_parent_len_norm_z_trans_EMI,self_child_len_norm_z_trans_EMI,self_submission_len_norm_z_trans_EMI,predictions_label_ft,predictions_score_ft,predictions_prob_disagree,predictions_ft
0,c7ecp8d,c7ecs2n,14m325,Ask_Politics,1355166551.0,karmanaut,ramblin_dash,karmanaut,0.053025,0.072843,...,0.514532,-0.116730,0.594431,0.624110,-0.011457,0.172262,0,0.6180,0.38180,no_disagreement
1,c7ecp8d,c7ecxpk,14m325,Ask_Politics,1355167049.0,zoolander951,ramblin_dash,karmanaut,0.040605,0.001874,...,0.514532,0.363800,0.594431,0.624110,0.012864,0.172262,0,0.7134,0.28640,no_disagreement
2,c7ecp8d,c7eflag,14m325,Ask_Politics,1355175187.0,zossima,ramblin_dash,karmanaut,0.037034,-0.026841,...,0.514532,0.571814,0.594431,0.624110,0.600814,0.172262,0,0.6580,0.34180,no_disagreement
3,c7efx8q,c7ejh1d,14m325,Ask_Politics,1355187644.0,fathermocker,senatorskeletor,karmanaut,0.046751,0.040446,...,0.269966,0.096978,0.594431,0.130100,0.070296,0.172262,1,0.9507,0.95070,disagree
4,c7efx8q,c7f8140,14m325,Ask_Politics,1355286254.0,pkmkii,senatorskeletor,karmanaut,0.029991,-0.043360,...,0.269966,0.644853,0.594431,0.130100,0.612353,0.172262,0,0.8580,0.14180,no_disagreement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627406,kfkh5ib,kfmd52k,18uirzt,politics,1970-01-01 00:00:01.703977996,knightforlife,freddiethebaer,optimalg,0.289315,0.222072,...,1.250681,0.671616,-0.580210,1.062374,0.498045,-0.857240,0,0.9550,0.04495,no_disagreement
1627407,kfklsb6,kforzbx,18uirzt,politics,1970-01-01 00:00:01.704023024,bakerfredricka,4blockhead,optimalg,0.192579,0.210383,...,1.367783,-0.043416,-0.580210,1.165189,-0.381238,-0.857240,0,0.9920,0.00774,no_disagreement
1627408,kfmq5kr,kfp6tzv,18uirzt,politics,1970-01-01 00:00:01.704032059,glassjar1,doubletfan,optimalg,0.078374,0.101258,...,-0.197753,-0.140261,-0.580210,0.242695,-0.122822,-0.857240,1,0.8500,0.85000,disagree
1627409,kfp3d4v,kfp8xh8,18uirzt,politics,1970-01-01 00:00:01.704033069,haulinboats,4blockhead,optimalg,0.137089,0.224533,...,0.600514,-0.617072,-0.580210,0.207855,-0.789836,-0.857240,0,0.9414,0.05875,no_disagreement


In [38]:
# merge scores

interactions_ideology

interactions_ideology = interactions_ideology.merge(user_scores, how='left', left_on='author_parent', right_on='User')
interactions_ideology = interactions_ideology.rename(columns={'Partisan_Score': 'parent_partisan'})
interactions_ideology = interactions_ideology.drop(columns=['User'])

# Merge child scores
interactions_ideology = interactions_ideology.merge(user_scores, how='left', left_on='author_child', right_on='User')
interactions_ideology = interactions_ideology.rename(columns={'Partisan_Score': 'child_partisan'})
interactions_ideology = interactions_ideology.drop(columns=['User'])

# Merge submission scores
interactions_ideology = interactions_ideology.merge(user_scores, how='left', left_on='author_submission', right_on='User')
interactions_ideology = interactions_ideology.rename(columns={'Partisan_Score': 'submission_partisan'})
interactions_ideology = interactions_ideology.drop(columns=['User'])

interactions_ideology

,msg_id_parent,msg_id_child,submission_id,subreddit,exact_time_child,author_child,author_parent,author_submission,finetuned_child_E_norm,finetuned_child_I_norm,...,self_parent_len_norm_z_trans_EMI,self_child_len_norm_z_trans_EMI,self_submission_len_norm_z_trans_EMI,predictions_label_ft,predictions_score_ft,predictions_prob_disagree,predictions_ft,parent_partisan,child_partisan,submission_partisan
0,c7ecp8d,c7ecs2n,14m325,Ask_Politics,1355166551.0,karmanaut,ramblin_dash,karmanaut,0.053025,0.072843,...,0.624110,-0.011457,0.172262,0,0.6180,0.38180,no_disagreement,0.017145,0.004449,0.004449
1,c7ecp8d,c7ecxpk,14m325,Ask_Politics,1355167049.0,zoolander951,ramblin_dash,karmanaut,0.040605,0.001874,...,0.624110,0.012864,0.172262,0,0.7134,0.28640,no_disagreement,0.017145,0.030790,0.004449
2,c7ecp8d,c7eflag,14m325,Ask_Politics,1355175187.0,zossima,ramblin_dash,karmanaut,0.037034,-0.026841,...,0.624110,0.600814,0.172262,0,0.6580,0.34180,no_disagreement,0.017145,-0.037449,0.004449
3,c7efx8q,c7ejh1d,14m325,Ask_Politics,1355187644.0,fathermocker,senatorskeletor,karmanaut,0.046751,0.040446,...,0.130100,0.070296,0.172262,1,0.9507,0.95070,disagree,0.013507,-0.063784,0.004449
4,c7efx8q,c7f8140,14m325,Ask_Politics,1355286254.0,pkmkii,senatorskeletor,karmanaut,0.029991,-0.043360,...,0.130100,0.612353,0.172262,0,0.8580,0.14180,no_disagreement,0.013507,-0.007571,0.004449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4933850,kfkh5ib,kfmd52k,18uirzt,politics,1970-01-01 00:00:01.703977996,knightforlife,freddiethebaer,optimalg,0.289315,0.222072,...,1.062374,0.498045,-0.857240,0,0.9550,0.04495,no_disagreement,-0.073672,0.013192,-0.024883
4933851,kfklsb6,kforzbx,18uirzt,politics,1970-01-01 00:00:01.704023024,bakerfredricka,4blockhead,optimalg,0.192579,0.210383,...,1.165189,-0.381238,-0.857240,0,0.9920,0.00774,no_disagreement,-0.030104,-0.066138,-0.024883
4933852,kfmq5kr,kfp6tzv,18uirzt,politics,1970-01-01 00:00:01.704032059,glassjar1,doubletfan,optimalg,0.078374,0.101258,...,0.242695,-0.122822,-0.857240,1,0.8500,0.85000,disagree,-0.024183,-0.019897,-0.024883
4933853,kfp3d4v,kfp8xh8,18uirzt,politics,1970-01-01 00:00:01.704033069,haulinboats,4blockhead,optimalg,0.137089,0.224533,...,0.207855,-0.789836,-0.857240,0,0.9414,0.05875,no_disagreement,-0.030104,0.033772,-0.024883


In [39]:
# merge scores

full_interactions_ideology

full_interactions_ideology = full_interactions_ideology.merge(user_scores, how='left', left_on='author_parent', right_on='User')
full_interactions_ideology = full_interactions_ideology.rename(columns={'Partisan_Score': 'parent_partisan'})
full_interactions_ideology = full_interactions_ideology.drop(columns=['User'])

# Merge child scores
full_interactions_ideology = full_interactions_ideology.merge(user_scores, how='left', left_on='author_child', right_on='User')
full_interactions_ideology = full_interactions_ideology.rename(columns={'Partisan_Score': 'child_partisan'})
full_interactions_ideology = full_interactions_ideology.drop(columns=['User'])

# Merge submission scores
full_interactions_ideology = full_interactions_ideology.merge(user_scores, how='left', left_on='author_submission', right_on='User')
full_interactions_ideology = full_interactions_ideology.rename(columns={'Partisan_Score': 'submission_partisan'})
full_interactions_ideology = full_interactions_ideology.drop(columns=['User'])

full_interactions_ideology

,msg_id_parent,msg_id_child,submission_id,subreddit,exact_time_child,author_child,author_parent,author_submission,finetuned_child_E_norm,finetuned_child_I_norm,...,self_parent_len_norm_z_trans_EMI,self_child_len_norm_z_trans_EMI,self_submission_len_norm_z_trans_EMI,predictions_label_ft,predictions_score_ft,predictions_prob_disagree,predictions_ft,parent_partisan,child_partisan,submission_partisan
0,c7ecp8d,c7ecs2n,14m325,Ask_Politics,1355166551.0,karmanaut,ramblin_dash,karmanaut,0.053025,0.072843,...,0.624110,-0.011457,0.172262,0,0.6180,0.38180,no_disagreement,0.017145,0.004449,0.004449
1,c7ecp8d,c7ecxpk,14m325,Ask_Politics,1355167049.0,zoolander951,ramblin_dash,karmanaut,0.040605,0.001874,...,0.624110,0.012864,0.172262,0,0.7134,0.28640,no_disagreement,0.017145,0.030790,0.004449
2,c7ecp8d,c7eflag,14m325,Ask_Politics,1355175187.0,zossima,ramblin_dash,karmanaut,0.037034,-0.026841,...,0.624110,0.600814,0.172262,0,0.6580,0.34180,no_disagreement,0.017145,-0.037449,0.004449
3,c7efx8q,c7ejh1d,14m325,Ask_Politics,1355187644.0,fathermocker,senatorskeletor,karmanaut,0.046751,0.040446,...,0.130100,0.070296,0.172262,1,0.9507,0.95070,disagree,0.013507,-0.063784,0.004449
4,c7efx8q,c7f8140,14m325,Ask_Politics,1355286254.0,pkmkii,senatorskeletor,karmanaut,0.029991,-0.043360,...,0.130100,0.612353,0.172262,0,0.8580,0.14180,no_disagreement,0.013507,-0.007571,0.004449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627406,kfkh5ib,kfmd52k,18uirzt,politics,1970-01-01 00:00:01.703977996,knightforlife,freddiethebaer,optimalg,0.289315,0.222072,...,1.062374,0.498045,-0.857240,0,0.9550,0.04495,no_disagreement,-0.073672,0.013192,-0.024883
1627407,kfklsb6,kforzbx,18uirzt,politics,1970-01-01 00:00:01.704023024,bakerfredricka,4blockhead,optimalg,0.192579,0.210383,...,1.165189,-0.381238,-0.857240,0,0.9920,0.00774,no_disagreement,-0.030104,-0.066138,-0.024883
1627408,kfmq5kr,kfp6tzv,18uirzt,politics,1970-01-01 00:00:01.704032059,glassjar1,doubletfan,optimalg,0.078374,0.101258,...,0.242695,-0.122822,-0.857240,1,0.8500,0.85000,disagree,-0.024183,-0.019897,-0.024883
1627409,kfp3d4v,kfp8xh8,18uirzt,politics,1970-01-01 00:00:01.704033069,haulinboats,4blockhead,optimalg,0.137089,0.224533,...,0.207855,-0.789836,-0.857240,0,0.9414,0.05875,no_disagreement,-0.030104,0.033772,-0.024883


In [18]:
print("Share retained full interactions:", len(full_interactions_ideology) / len(EMI_df))
print("Share retained parent & child interactions:", len(interactions_ideology) / len(EMI_df))

Share retained full interactions: 0.3295152024660221
Share retained parent & child interactions: 0.9989979355325702


In [19]:
print("Share retained pure texts:", len(texts_ideology) / len(texts_EMI))


Share retained pure texts: 0.9990534957315393


## **SAVE**

In [40]:
texts_ideology.to_csv(f"../data/analysis_data/texts_ideology_{scores}.csv", index = False)


# be mindful: for comments an arbitraty label is taken (first one?)
# this is not representative anymore

In [21]:
full_interactions_ideology.to_csv(f"../data/analysis_data/full_interactions_ideology_{scores}.csv", index = False)

# incl submission partsian = human discussion

In [22]:
interactions_ideology.to_csv(f"../data/analysis_data/interactions_ideology_{scores}.csv", index = False)

# parent & child with score --> incl. mods without a score

In [39]:
user_scores[user_scores["User"] == "politicsmod"]

,User,Partisan_Score
